## GPU计算
到目前为止，我们一直在使用CPU计算。对复杂的神经网络和大规模的数据来说，使用CPU来计算可能不够高效。在本节中，我们将介绍如何使用单块NVIDIA GPU来计算。所以需要确保已经安装好了PyTorch GPU版本。准备工作都完成后，下面就可以通过nvidia-smi命令来查看显卡信息了。

In [1]:
!nvidia-smi  # 对Linux/macOS用户有效

Tue Dec 10 11:04:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 430.50       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:89:00.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |     22MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:8A:00.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

### 计算设备
$PyTorch$可以指定用来存储和计算的设备，如使用内存的$CPU$或者使用显存的$GPU$。默认情况下，$PyTorch$会将数据创建在内存，然后利用$CPU$来计算。

In [2]:
import torch
from torch import nn

torch.cuda.is_available()

True

查看$GPU$数量

In [3]:
torch.cuda.device_count()

8

查看当前$GPU$索引号，索引号从$0$开始：

In [4]:
torch.cuda.get_device_name(0)

'Tesla V100-PCIE-32GB'

### Tensor的GPU计算
默认情况下，$Tensor$会被存在内存上。因此，之前我们每次打印$Tensor$的时候看不到$GPU$相关标识。

In [5]:
x = torch.tensor([1, 2, 3])
x

tensor([1, 2, 3])

使用$.cuda()$可以将$CPU$上的$Tensor$转换（复制）到$GPU$上。如果有多块$GPU$，我们用$.cuda(i)$来表示第 $i$ 块GPU及相应的显存（$i$从$0$开始）且$cuda(0)$和$cuda()$等价。

In [6]:
x = x.cuda(7)
x

tensor([1, 2, 3], device='cuda:7')

In [7]:
x.device

device(type='cuda', index=7)

我们可以直接在创建的时候就指定设备。

In [8]:
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

x = torch.tensor([1,2,3], device=device)

x = torch.tensor([1,2,3]).to(device)
x

tensor([1, 2, 3], device='cuda:7')

如果对在$GPU$上的数据进行运算，那么结果还是存放在$GPU$上。

In [9]:
y = x**2
y

tensor([1, 4, 9], device='cuda:7')

需要注意的是，**存储在不同位置中的数据是不可以直接进行计算的**。即存放在CPU上的数据不可以直接与存放在GPU上的数据进行运算，位于不同GPU上的数据也是不能直接进行计算的。

In [10]:
z = y + x.cpu()

RuntimeError: expected device cuda:7 but got device cpu

### 模型的GPU计算
同Tensor类似，PyTorch模型也可以通过.cuda转换到GPU上。我们可以通过检查模型的参数的device属性来查看存放模型的设备。

In [11]:
net = nn.Linear(3, 1)
list(net.parameters())[0].device

device(type='cpu')

可见模型在CPU上，将其转换到GPU上:

In [12]:
net.cuda(7)
list(net.parameters())[0].device

device(type='cuda', index=7)

同样的，我么需要保证模型输入的Tensor和模型都在同一设备上，否则会报错。

In [13]:
x = torch.rand(2, 3).cuda(7)
net(x)

tensor([[0.4866],
        [0.3854]], device='cuda:7', grad_fn=<AddmmBackward>)

+ PyTorch可以指定用来存储和计算的设备，如使用内存的CPU或者使用显存的GPU。在默认情况下，PyTorch会将数据创建在内存，然后利用CPU来计算。
+ PyTorch要求计算的所有输入数据都在内存或同一块显卡的显存上。